In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
datos = pd.read_csv('/content/drive/MyDrive/modulo 6/AAME.csv')
datos

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,17-03-1980,3.85,0.00,15000,4.05,3.85,3.519899
1,18-03-1980,3.80,0.00,10250,4.00,3.80,3.474187
2,19-03-1980,3.85,0.00,33500,4.05,3.85,3.519899
3,20-03-1980,3.80,0.00,8750,4.00,3.80,3.474187
4,21-03-1980,3.75,0.00,12750,3.95,3.75,3.428474
...,...,...,...,...,...,...,...
10773,06-12-2022,2.66,2.70,2700,2.70,2.66,2.660000
10774,07-12-2022,2.70,2.73,700,2.73,2.70,2.700000
10775,08-12-2022,2.52,2.62,13800,2.69,2.59,2.590000
10776,09-12-2022,2.53,2.57,3600,2.58,2.57,2.570000
